In [1]:
import pandas as pd
import numpy as np
import sklearn
from sklearn.utils import Bunch
from boostaroota import BoostARoota
import random
from itertools import chain
import datetime,zlib,zipfile,os

class BoostARootaRes:
	def __init__(self,dataframe,num=100,random_state=0):
		self.input=dataframe
		self.loop=num
		assert len(self.input)>120,"Dataset is too small, please check again."
		self.min_freq=5 if int(40/len(self.input)*100)<5 else int(40/len(self.input)*100)
		self.random_frac=list(np.linspace(self.min_freq,self.min_freq*5 if self.min_freq<11 else 51,self.loop)/100)
		self.random_state_bias=random_state
	def get_df(self,zip_action=True):
		self.zip_action=zip_action
		return self._df_generator(self.input,self.loop)
	def _df_generator(self,dataframe,num):
		timestamp=datetime.datetime.timestamp(datetime.datetime.now())
		with zipfile.ZipFile(f"G1_Folds_{timestamp}.zip", 'w') as myzip:
			for i in range(num):
				new_data=dataframe.sample(frac=self.random_frac[i],random_state=i+self.random_state_bias)
				new_data.to_parquet(f"{self.random_frac[i]}_{i}_{timestamp}.parquet.gzip", compression="gzip")
				if self.zip_action:
					myzip.write(f"{self.random_frac[i]}_{i}_{timestamp}.parquet.gzip")
					os.remove(f"{self.random_frac[i]}_{i}_{timestamp}.parquet.gzip")
				yield new_data.iloc[:,:-1],new_data.iloc[:,-1]
	def get_feature_list(self,X:pd.DataFrame,y:pd.DataFrame):
		br=BoostARoota(metric='logloss',cutoff=1,iters=100,max_rounds=100)
		br.fit(X,y)
		return br.keep_vars_.tolist()

In [2]:
from sklearn.datasets import load_breast_cancer
cancer=load_breast_cancer()
y=cancer['target']
X=cancer['data']
df=dict(list(zip(cancer['feature_names'],X.T)))
df.update({'TARGET':y})
df=pd.DataFrame(df)
BARR=BoostARootaRes(df,3,123)
total_list=list(chain(*[BARR.get_feature_list(X,y) for X,y in BARR.get_df()]))
final_list=list(set(total_list))
final_df=df[final_list]

[18:13:08] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:576: 
Parameters: { "silent" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


Round:  1  iteration:  1
[18:13:08] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:576: 
Parameters: { "silent" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


Round:  1  iteration:  2
[18:13:08] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:576: 
Parameters: { "sil